## First contact with tensorflow


More resources:

- Tensorflow for deep learning research: https://web.stanford.edu/class/cs20si/
- Tensorflow book: https://github.com/nfmcclure/tensorflow_cookbook
- convnet example in tensorflow: http://www.jessicayung.com/explaining-tensorflow-code-for-a-convolutional-neural-network/
- nlp course https://cs224d.stanford.edu/lectures/CS224d-Lecture7.pdf
- Structuring Your TensorFlow Models https://danijar.com/structuring-your-tensorflow-models/
- TensorFlow Tutorials with YouTube Videos: https://github.com/Hvass-Labs/TensorFlow-Tutorials
- http://learningtensorflow.com/lesson10/

#### Working with big image datasets

- https://kwotsin.github.io/tech/2017/02/11/transfer-learning.html

In [1]:
import tensorflow as tf
import numpy as np

np.__version__, tf.__version__

('1.13.0', '1.2.0')

### Key idea in tensorflow

Working in tensorflow involves almost allways two steps

- 1: assemble a graph
- 2: use a session to execute operations in the graph

##### Graph interpretation

- Nodes correspond to  operators, variables, and constants
- Edges correspond to tensors

For example, we can build a neural net (which is a graph) and then create a session and train the model.

##### Using graphs in sessions

In order to make graphs usefull we have to initialize a tf.session. Tensorflow sessions are assigned to python variables so that they can be used afterwards. **A Session object encapsulates the environment in which Operation objects are executed, and Tensor objects are evaluated.**

Let us imagine that we have defined the output of a neural net in tensorflow and we want to get the class predictions for a given input image `x`. We could write something like:

```python
session_net = tf.session()
session_net.run(output_neural_net, feed_dict={x:mnist_image} )
```
To get the numerical values of the output

##### Let us begin with a very simple graph, adding two floats


In the following code code, ``3`` and ``5`` are concrete Float values. The variable `a` is a tensor who is not evaluated,
to get its value you need to define a session and use the `run` method.

Given a session `s` the method `s.run` requires an argument `fetches` which is a graph (we will see more complicated stuff later).
The function has other possible inputs which are by default `feed_dict=None, options=None, run_metadata=None`.



In [2]:
a = tf.add(3, 5)

In [3]:
a

<tf.Tensor 'Add:0' shape=() dtype=int32>

In [4]:
toy_sess = tf.Session()
toy_sess.run(a)

8

#### Defining a symbolic graph

Usually graphs are defined in a symbolic way. For example, a graph can be a machine learning model, which is fed by data to produce an output value.
In this scenario we will use the `feed_dict` argument in the `run` method.

##### Example

In the following code code, ``a`` and ``b`` are symbolic variables called placeholders.

A symbolic operation `c = a+b` (or `c = tf.add(a,b)`)  is defined using the symbolic variables ``a`` and ``b``.


In [5]:
a = tf.placeholder("float")
b = tf.placeholder("float")

In [6]:
type(a)

tensorflow.python.framework.ops.Tensor

In [7]:
c = a + b
# we can also write
#c = tf.add(a,b)

In [8]:
type(c)

tensorflow.python.framework.ops.Tensor


In order to evaluate `c` we have to provide values for the inputs ``a`` and ``b`` using feed_dict.




In [9]:
toy_sess = tf.Session()

In [10]:
toy_sess.run(c, feed_dict={a:10, b:5})

15.0

In [11]:
toy_sess.close()


It is important to close a session to avoid unwanted changes in a graph. 
If we now redefine, by mistake, c = a*b we will modify c.

##### Try using  `with`  always when running a session,

The following code will automatically close the session outside the with statement

```python
with tf.Session() as sess:
    print(sess.run(a))
```

In [12]:
with tf.Session() as sess:
    print(sess.run(c, feed_dict={a:10, b:5}))

15.0


#### Even simpler example



In [13]:
a = tf.add(3, 5)
toy_sess = tf.Session()
toy_sess.run(a)

8

In [14]:
# Notice that if we print `a` we don't get 8. We get a tensor
print(a)

Tensor("Add_1:0", shape=(), dtype=int32)


## Basic tensor operations

- tf.add
- tf.substract
- tf.multiply
- tf.matmul

Using tensorflow operations  we define an abstract graph which is then fed with concrete values using `feed_dict`.


In [15]:
sess = tf.Session()

In [16]:
type(sess)

tensorflow.python.client.session.Session

In [18]:
y = tf.add(a,b)
sess.run(y, feed_dict={a:10., b:5.}), sess.run(y, feed_dict={a:10., b:50.})

TypeError: Input 'y' of 'Add' Op has type float32 that does not match type int32 of argument 'x'.

In [ ]:
y = tf.subtract(a,b)
sess.run(y, feed_dict={a:10, b:5}), sess.run(y, feed_dict={a:10, b:50})

In [ ]:
y = tf.multiply(a,b)
sess.run(y, feed_dict={a:10, b:5}), sess.run(y, feed_dict={a:10, b:50})

#### A graph combining two operations



In [ ]:
x = 2
y = 3
op1 = tf.add(x, y)
op2 = tf.multiply(x, y)
useless = tf.multiply(x, op1)
op3 = tf.pow(op2, op1)

with tf.Session() as sess:
    print("result of the computation:", sess.run(op3))
               
# This will print (2*3)^(2+3) = 6**5
6**5

Notice that in the previous code the tensor `useless` was not used at all. The `session.run` method was used to fetch the value of `op3` which does not depend on `useless`. We can, nevertheless, open a new session and retrieve its value

In [ ]:
with tf.Session() as sess:
    print("result of the computation of useless:", sess.run(useless))

We can fetch more than a single node in the graph, we just need to passs to the `run` method a list of the nodes that we want to fetch instead of a single one

In [ ]:
x = 2
y = 3
op1 = tf.add(x, y)
op2 = tf.multiply(x, y)
useless = tf.multiply(x, op1)
op3 = tf.pow(op2, op1)

with tf.Session() as sess:
    op3, not_useless = sess.run([op3, useless])
    print("opt3 value: ", op3,  "\nnot_useless value", not_useless)

##### Same example with symbolic inputs

We can use symbolic inputs to define a single time the graph which we can use for as many values as we want

In [ ]:
x = tf.placeholder("float")
y = tf.placeholder("float")
op1 = tf.add(x, y)
op2 = tf.multiply(x, y)
useless = tf.multiply(x, op1)
op3 = tf.pow(op2, op1)

with tf.Session() as sess:
    op3_value, not_useless_value = sess.run([op3, useless], feed_dict={x:2, y:3})
    print("opt3 value: ",op3_value,  "\nnot_useless value", not_useless_value)

In [ ]:
with tf.Session() as sess:
    op3_value, not_useless_value = sess.run([op3, useless], feed_dict={x:3, y:3})
    print("opt3 value: ",op3_value,  "\nnot_useless value", not_useless_value)

## How to properly define a symbolic graph

We will define a graph using  `tf.Graph()`.

Using the `with` statement as follows

```python
g = tf.Graph()
with g.as_default():
    # define tensors here, and the graph 
```

The object `tf.Graph()` will keep the tensors, operations, relations defined inside the `with` block.
This is very usefull for defining something and keeping it in an object.

We can later on initialize the session with a particular graph and make computations with the `run` method.

Notice that tensorflow is build to use one graph in one session.


In [19]:
g_nonsense = tf.Graph()
with g_nonsense.as_default():
    2

In [20]:
sess = tf.Session(graph=g_nonsense)

In [21]:
sess.graph_def

versions {
  producer: 22
}

In [26]:
tf.reset_default_graph()

g2 = tf.Graph()
with g2.as_default():
    x = tf.add(2,3) # This x is saved in g2 

In [27]:
# We can use g2 previously defined, import a session with the graph g2 and make computations
# In this example we fetch the value of x
with tf.Session(graph=g2) as sess:
    print("value of x:", sess.run(x))

value of x: 5


In [28]:
sess = tf.Session(graph=g2)

In [29]:
sess.graph_def

node {
  name: "Add/x"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 2
      }
    }
  }
}
node {
  name: "Add/y"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 3
      }
    }
  }
}
node {
  name: "Add"
  op: "Add"
  input: "Add/x"
  input: "Add/y"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
versions {
  producer: 22
}

## Summing up

- **Graphs**: Graphs in Tensorflow are just a map / path that the computation will take. It does not hold any values and does not execute anything.

- **Session**: on the other hand, session needs a graph, data and a run-time to execute. This concept of Graphs and Sessions lets TensorFolow separate the flow definitions or models from the actual computation runtime.

- **Separating the run-time from flow graph**:  This was, most probably, done to separate out the graph definition from the run-time configurations and actual execution with data. E.g, the run-time can be on a cluster. So each of the execution run-times in the cluster needs to have the same definition of the graph. But each of the run-time might locally have a different set of data during the execution process. So it is important that input and output data can be supplied during the distributed execution in the cluster.

- ** Why Placeholders and Not Variables**: Placeholders act as input / output conduits for the Graphs. If you visualize your graph as a number of nodes - placeholders are input or output nodes.

Real question is why does TensorFlow not use a normal variable for the I/O nodes? Why have another type?

During the training process (when the program is executing in a session), it needs to be ensured that actual values are used to train a model. Basically feed_dict inside a training process would accept only actual values, e.g. a Numpy ndarry. These actual values can not be supplied by a TensorFlow variable, as Tensorflow variables do not have data unless eval() or session.run() is used. However the training statement itself is part of a session.run() function - Therefore it can not take another session.run() inside it to resolve the tensor variable to data. By this time, a session.run() already has to bind to a specific run time configuration and data. 